In [14]:
import pandas as pd
from datetime import date

# Load reconciliation base (for demo: join orders + refunds + PSP + GL)
orders = pd.read_csv("../data/orders.csv")
refunds = pd.read_csv("../data/refunds.csv")
psp = pd.read_csv("../data/psp_settlements.csv")
gl = pd.read_csv("../data/gl_entries.csv")

orders = orders.rename(columns={
    "net_amount": "expected_net"
})

gl = gl.rename(columns={
    "amount": "gl_amount"
})
# Normalize column names
orders.columns = orders.columns.str.strip().str.lower()
gl.columns = gl.columns.str.strip().str.lower()

# Rename for consistency
orders = orders.rename(columns={"net_amount": "expected_net"})
gl = gl.rename(columns={"reference": "order_id"})

# Compute signed GL amount
# Ensure numeric types for accounting amounts
gl["debit"] = pd.to_numeric(gl["debit"], errors="coerce")
gl["credit"] = pd.to_numeric(gl["credit"], errors="coerce")
gl["signed_amount"] = gl["debit"].fillna(0) - gl["credit"].fillna(0)

# Aggregate GL to order level
gl_order = (
    gl.groupby("order_id", as_index=False)
      .agg(gl_amount=("signed_amount", "sum"))
)

# Merge Orders → GL
merged = orders.merge(gl_order, on="order_id", how="left")

# Reconciliation difference
merged["variance"] = merged["expected_net"] - merged["gl_amount"]

# Identify exceptions
exceptions = merged[
    merged["gl_amount"].isna() | (merged["variance"].abs() > 1)
]

# Build exception report
report = pd.DataFrame({
    "report_date": date.today(),
    "exception_type": "GL_MISMATCH",
    "order_id": exceptions["order_id"],
    "variance": exceptions["variance"],
    "status": "OPEN",
    "priority": "HIGH"
})

# Write output
report.to_csv("../output/daily_exception_report.csv", index=False)

print(f"Generated {len(report)} exceptions")


Generated 13 exceptions


In [10]:
print("ORDERS columns:", list(orders.columns))
print("PSP columns:", list(psp.columns))
print("GL columns:", list(gl.columns))

ORDERS columns: ['order_id', 'order_date', 'customer_id', 'product_id', 'quantity', 'unit_price', 'gross_amount', 'tax_amount', 'expected_net', 'status', 'payment_method']
PSP columns: ['settlement_id', 'settlement_date', 'payment_method', 'transaction_count', 'gross_settlement', 'fees', 'net_settlement', 'settlement_period_start', 'settlement_period_end']
GL columns: ['gl_entry_id', 'entry_date', 'account_code', 'account_name', 'debit', 'credit', 'description', 'reference']
